<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Mon_premier_reseau_de_neurones_avec_Keras-Kleiber.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Mon premier réseau de neurones avec Keras

## Approximer une étrange loi de la nature directement à partir de données

Vous allez créer votre premier réseau de neurones vraiment très simple pour faire de la reconnaissance de formes (pattern matching).

Les biologistes ont découvert que certaines caractéristiques des êtres vivants peuvent être décrites par des lois de puissance. Par exemple, dans la nature, il y a plus de souris que d'éléphants et plus de mouches que de souris.  La taille des populations animales suit donc une loi de puissance avec la masse corporelle comme variable explicative.  Il en va de même pour le pouls, la durée de vie, le nombre d'œufs pondus par un oiseau, etc. Parfois, ces lois de puissance peuvent être prédites à partir de la géométrie, mais parfois elles demeurent mystérieuses. 

Par exemple, la loi de Kleiber, formulée par le biologiste Max Kleiber dans les années 1930, postule que la consommation d'énergie (le métabolisme) des animaux, y compris les humains, varie comme la puissance 3/4 de leur masse corporelle. Cette loi fonctionne des bactéries jusqu'aux baleines mais demeure pour le moment sans explication physique ou géométrique satisfaisante. 

https://fr.wikipedia.org/wiki/Loi_de_Kleiber

On peut légitimenent se questionner sur l'utilité d'un tel exercice? En fait, ce qui est «intéressant» c'est de constater que le réseau de neurones va apprendre «seul» à partir des données à approximer cette étrange loi de la nature. 

**Note**: Il n'est pas important de comprendre le détail du code informatique pour le moment. Ne vous inquiétez pas, des explications détaillées suivront bientôt.


# Acquisition des données...

### Source des données: 
http://sites.science.oregonstate.edu/~schaferd/Sleuth/data-sets.html

Le **Fichier ex0826.csv** contenu dans l'archive **sleuth3csv.zip** a été renommé **LoiDeKleiber.csv** avec une entête en français

Ramsey, F., & Schafer, D. (2012). The statistical sleuth: a course in methods of data analysis. Cengage Learning.


In [1]:
! mkdir DATA

In [2]:
! wget "https://github.com/ClaudeCoulombe/VIARENA/blob/master/DATA/LoiDeKleiber.csv?raw=True" -O DATA/LoiDeKleiber.csv

--2021-11-25 05:58:44--  https://github.com/ClaudeCoulombe/VIARENA/blob/master/DATA/LoiDeKleiber.csv?raw=True
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ClaudeCoulombe/VIARENA/raw/master/DATA/LoiDeKleiber.csv [following]
--2021-11-25 05:58:44--  https://github.com/ClaudeCoulombe/VIARENA/raw/master/DATA/LoiDeKleiber.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ClaudeCoulombe/VIARENA/master/DATA/LoiDeKleiber.csv [following]
--2021-11-25 05:58:44--  https://raw.githubusercontent.com/ClaudeCoulombe/VIARENA/master/DATA/LoiDeKleiber.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

# Quelques lignes de code...

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd


loi_kleiber_data = pd.read_csv("DATA/LoiDeKleiber.csv")
loi_kleiber_data.head()

# masse en kg
# métabolisme en kJ par jour

,NomCommun,Espece,Masse,Metabolisme,DureeDeVie
0,Echidna,Tachiglossus aculeatus,2.500,302.0,14.0
1,Long-beaked echidna,Zaglossus bruijni,10.300,594.0,20.0
2,Platypus,Ornithorhynchus anatinus,1.300,229.0,9.0
3,Opossum,Lutreolina crassicaudata,0.812,196.0,5.0
4,South American opossum,Didelphis marsupialis,1.330,299.0,6.0


In [18]:
from sklearn.preprocessing import StandardScaler

variable_explicative = loi_kleiber_data['Masse'].values.reshape(-1, 1)
scaler_var_expl = StandardScaler()
scaler_var_expl.fit(variable_explicative)
variable_explicative = scaler_var_expl.transform(variable_explicative)

variable_dependante = loi_kleiber_data['Metabolisme'].values.reshape(-1, 1)
scaler_var_dep = StandardScaler()
scaler_var_dep.fit(variable_dependante)
variable_dependante = scaler_var_dep.transform(variable_dependante)

reseau_de_neurones = tf.keras.models.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1]),
                                                 tf.keras.layers.Dense(units=10, activation='relu'),
                                                 tf.keras.layers.Dense(units=1)])
reseau_de_neurones.summary()
reseau_de_neurones.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='mean_squared_error')
history = reseau_de_neurones.fit(variable_explicative,variable_dependante,epochs=500,verbose=1)
history.history['loss'][-1]


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1)                 2         
                                                                 
 dense_4 (Dense)             (None, 10)                20        
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
3/3 [==============================] - 0s 4ms/step - loss: 0.8776
Epoch 2/500
3/3 [==============================] - 0s 4ms/step - loss: 0.8698
Epoch 3/500
3/3 [==============================] - 0s 7ms/step - loss: 0.8641
Epoch 4/500
3/3 [==============================] - 0s 3ms/step - loss: 0.8562
Epoch 5/50

0.007126966957002878

Nous allons valider la fonction que notre réseau a apprise entre la masse corporelle sur des données de test qui ne faisaient pas partie du jeu de données d'entraînement.

In [44]:
# Être humain, Homo sapiens
animal = "être humain"
masse = 6.50E+01
masse_normalisee = scaler_var_expl.transform(np.array([masse]).reshape(-1, 1))
vraie_valeur_metabolisme = 7.56E+03
print("\nAnimal:",animal,
      ", Masse:",masse,
      ", prédiction =>",
      "Métabolisme:",scaler_var_dep.inverse_transform(reseau_de_neurones.predict(masse_normalisee))[0][0],
      "Vraie valeur du métabolisme :",vraie_valeur_metabolisme)



Animal: être humain , Masse: 65.0 , prédiction => Métabolisme: 7270.268 Vraie valeur du métabolisme : 7560.0


In [45]:
# Chat, Felis silvestris, 3.00E+00, 5.46E+02, 11
animal = "chat"
masse = 3.00E+00
masse_normalisee = scaler_var_expl.transform(np.array([masse]).reshape(-1, 1))
vraie_valeur_metabolisme = 5.46E+02
print("\nAnimal:",animal,
      ", Masse:",masse,
      ", prédiction =>",
      "Métabolisme:",scaler_var_dep.inverse_transform(reseau_de_neurones.predict(masse_normalisee))[0][0],
      "Vraie valeur du métabolisme :",vraie_valeur_metabolisme)




Animal: chat , Masse: 3.0 , prédiction => Métabolisme: 606.13947 Vraie valeur du métabolisme : 546.0


In [46]:
# Cheval, Equus cabalus, 4.00E+02, 3.20E+04, 40
animal = "cheval"
masse = 4.00E+02
masse_normalisee = scaler_var_expl.transform(np.array([masse]).reshape(-1, 1))
vraie_valeur_metabolisme = 3.20E+04
print("\nAnimal:",animal,
      ", Masse:",masse,
      ", prédiction =>",
      "Métabolisme:",scaler_var_dep.inverse_transform(reseau_de_neurones.predict(masse_normalisee))[0][0],
      "Vraie valeur du métabolisme :",vraie_valeur_metabolisme)



Animal: cheval , Masse: 400.0 , prédiction => Métabolisme: 30928.793 Vraie valeur du métabolisme : 32000.0


### Conclusion

Vous pouvez constater que le réseau de neurones retourne des valeurs assez proches des vraies valeurs données par la loi de Kleiber. Il est important de comprendre que le réseau de neurones n'apprend pas une formule exacte mais bien qu'il calcule itérativement une fonction qui approxime cette formule.  

Choses à retenir:

* Le réseau de neurones est capable d'apprendre à approximer une fonction directement à partir des données
* Le processus d'apprentissage est itératif
    